In [98]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import keras_preprocessing

In [99]:
traindf=pd.read_csv("train.csv")
testdf=pd.read_csv("test.csv")
def append_ext(fn):
    return str(fn)+".png"

traindf["id"]=traindf["id"].apply(append_ext)
testdf["id"]=testdf["id"].apply(append_ext)
traindf["label"]=traindf["label"].apply(str)

In [112]:
from keras_preprocessing.image import ImageDataGenerator
datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.25)

batch_size = 50
img_height, img_width = 28, 28  # Adjust based on your image size

train_generator=datagen.flow_from_dataframe(
dataframe=traindf,
directory="train",
x_col="id",
y_col="label",
subset="training",
batch_size=50,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(32,32))

valid_generator=datagen.flow_from_dataframe(
dataframe=traindf,
directory="train",
x_col="id",
y_col="label",
subset="validation",
batch_size=50,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(32,32))

test_datagen=ImageDataGenerator(rescale=1./255.)
test_generator=test_datagen.flow_from_dataframe(
dataframe=testdf,
directory="test",
x_col="id",
y_col=None,
batch_size=50,
seed=42,
shuffle=False,
class_mode=None,
target_size=(32,32))


Found 45000 validated image filenames belonging to 10 classes.
Found 15000 validated image filenames belonging to 10 classes.
Found 10000 validated image filenames.


In [113]:
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),
    #layers.Conv2D(128, (3, 3), activation='relu'),
    #layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.25),
    layers.Dense(10, activation='softmax')
])


In [114]:
opt = keras.optimizers.Adam(learning_rate=0.001)
#opt = keras.optimizers.RMSprop(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
#model.compile(optimizer=rmsprop(lr=0.0001, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])

In [115]:
model.fit(train_generator,
          steps_per_epoch=200,
          epochs=10,
          validation_data=valid_generator,
          validation_steps=200,
)

Epoch 1/10
200/200 [==============================] - 39s 188ms/step - loss: 0.8356 - accuracy: 0.6959 - val_loss: 0.5328 - val_accuracy: 0.8013
Epoch 2/10
200/200 [==============================] - 31s 154ms/step - loss: 0.5508 - accuracy: 0.7989 - val_loss: 0.4394 - val_accuracy: 0.8387
Epoch 3/10
200/200 [==============================] - 27s 135ms/step - loss: 0.4831 - accuracy: 0.8192 - val_loss: 0.4005 - val_accuracy: 0.8579
Epoch 4/10
200/200 [==============================] - 25s 126ms/step - loss: 0.4331 - accuracy: 0.8431 - val_loss: 0.3692 - val_accuracy: 0.8659
Epoch 5/10
200/200 [==============================] - 26s 129ms/step - loss: 0.4082 - accuracy: 0.8524 - val_loss: 0.3610 - val_accuracy: 0.8660
Epoch 6/10
200/200 [==============================] - 24s 122ms/step - loss: 0.3851 - accuracy: 0.8545 - val_loss: 0.3383 - val_accuracy: 0.8740
Epoch 7/10
200/200 [==============================] - 25s 126ms/step - loss: 0.3669 - accuracy: 0.8642 - val_loss: 0.3240 - val_ac

In [116]:
model.evaluate(valid_generator, steps=200)

200/200 [==============================] - 17s 83ms/step - loss: 0.3040 - accuracy: 0.8847


[0.30395662784576416, 0.8847000002861023]

In [117]:
test_generator.reset()
pred=model.predict(test_generator,
steps=200,
verbose=1)

200/200 [==============================] - 17s 84ms/step


In [119]:
predicted_class_indices=np.argmax(pred,axis=1)
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
filenames=test_generator.filenames
print(pred.shape)
results=pd.DataFrame({"id":filenames,
                      "label":predictions})
results.to_csv("results.csv",index=False)

(10000, 10)
